In [1]:
# import libraries

import pandas as pd
import numpy as np
import requests
from http import client
import getpass 
from time import sleep
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
# define secret key 
mykey = getpass.getpass('INPUT KEY: ')
rapidkey = getpass.getpass('RAPID API KEY:')

In [3]:
# assign selected endpoint urls for quick use in function calls later
games_url =  f"https://rawg-video-games-database.p.rapidapi.com/games?key={mykey}"
developers_url = f"https://rawg-video-games-database.p.rapidapi.com/developers?key={mykey}"
platforms_url = f"https://rawg-video-games-database.p.rapidapi.com/platforms?key={mykey}"
genres_url = f"https://rawg-video-games-database.p.rapidapi.com/genres?key={mykey}"
tags_url = f"https://rawg-video-games-database.p.rapidapi.com/tags?key={mykey}"

# assign selected urls for quick use in function calls later.
headers = {'x-rapidapi-host': "rawg-video-games-database.p.rapidapi.com",'x-rapidapi-key': rapidkey}


In [4]:
# define a function to request json data from rawg API returning data for selected pages
def rawg_getter(baseurl,headers, x):
    params = {'page':x, 'page_size':40}
    response = requests.get(baseurl, headers=headers,params = params)
    r = response.json()
    
    return r['results']

In [5]:
# collect data on games from the first 5000 pages
games = list()
for x in range(1,5001):
    games = games + rawg_getter(games_url,headers,x)

len(games)

200000

In [10]:
# collect data on developers from first 101 pages
developers = list()
for x in range(1,101):
    developers = developers + rawg_getter(developers_url,headers,x)

len(developers)

4000

In [7]:
#collect data on platforms from the first 10 pages
platforms = list()
for x in range(1,3):
    platforms = platforms + rawg_getter(platforms_url, headers, x)

len(platforms)

51

In [8]:
# collect data on genres from the first 10 pages
genres = list()
for x in range(1,2):
    genres = genres + rawg_getter(genres_url, headers, x)

len(genres)


19

In [9]:
# collect game tags from the first 10 pages
tags = list()
for x in range (1,20):
    tags = tags + rawg_getter(tags_url, headers,x)

## Extract desirable data to create a usable dataframe 
From games the information we want will be:
- id
- slug
- name
- released
- tba
- rating
- rating_top
- metacritic
- playtime
- platforms[0]['platform']['name']
- platforms[1]['requirements']['minimum']
- platforms[1]['requirements']['recommended']
- esrb_rating['name']

In [11]:
game_data = []
# define a function to collect the data from games
def game_data_getter(json_data, index = 0):
    data = list()
    data.append({'id':json_data[index]['id'],
    'title':json_data[index]['name'],
    'release_date':json_data[index]['released'],
    'tba':json_data[index]['tba'],
    'rating/5':json_data[index]['rating'],
    'top_rating':json_data[index]['rating_top'],
    'metacritic_score':json_data[index]['metacritic'],
    'average_length':json_data[index]['playtime'],
    'platforms': json_data[index]['platforms'],
    'genre':json_data[index]['genres'],
    'esrb_rating':json_data[index]['esrb_rating']})
    return data[0]

In [12]:
# create loop to populate a list of lists
games_list = []
for g in range(len(games)):
    games_list.append(game_data_getter(games,g))

len(games_list)
    

200000

In [13]:
games_df = pd.DataFrame(games_list)
games_df.head(20)

,id,title,release_date,tba,rating/5,top_rating,metacritic_score,average_length,platforms,genre,esrb_rating
0,3498,Grand Theft Auto V,2013-09-17,False,4.48,5,97.0,71,"[{'platform': {'id': 187, 'name': 'PlayStation...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}"
1,3328,The Witcher 3: Wild Hunt,2015-05-18,False,4.67,5,92.0,49,"[{'platform': {'id': 7, 'name': 'Nintendo Swit...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}"
2,4200,Portal 2,2011-04-18,False,4.62,5,95.0,11,"[{'platform': {'id': 14, 'name': 'Xbox 360', '...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","{'id': 2, 'name': 'Everyone 10+', 'slug': 'eve..."
3,5286,Tomb Raider (2013),2013-03-05,False,4.06,4,86.0,11,"[{'platform': {'id': 5, 'name': 'macOS', 'slug...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}"
4,5679,The Elder Scrolls V: Skyrim,2011-11-11,False,4.43,5,94.0,45,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}"
5,12020,Left 4 Dead 2,2009-11-17,False,4.08,4,89.0,9,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}"
6,4291,Counter-Strike: Global Offensive,2012-08-21,False,3.58,4,81.0,62,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}"
7,13536,Portal,2007-10-09,False,4.52,5,90.0,4,"[{'platform': {'id': 21, 'name': 'Android', 's...","[{'id': 3, 'name': 'Adventure', 'slug': 'adven...","{'id': 3, 'name': 'Teen', 'slug': 'teen'}"
8,802,Borderlands 2,2012-09-18,False,4.04,4,89.0,10,"[{'platform': {'id': 18, 'name': 'PlayStation ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}"
9,4062,BioShock Infinite,2013-03-26,False,4.39,5,94.0,12,"[{'platform': {'id': 14, 'name': 'Xbox 360', '...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}"


In [14]:
platforms = games_df['platforms']
# define a function to strip platform names from platform dictionary.
def platform_names(platform_list):
    platform_name_list = []
    for platform in platform_list:
        platform_name_list.append(platform['platform']['name'])
        
    return platform_name_list

games_df['platforms'] = platforms.apply(lambda x : platform_names(x))

In [15]:
# define a function to replace genres with genre names and create a new column for genre_ids
def genre_id(genres_list):
    genre_ids_list = []
    for genre in genres_list:
        genre_ids_list.append(genre['id'])
    return genre_ids_list

games_df['genre_ids'] = games_df['genre'].apply(lambda x: genre_id(x))

def genre_names(genres_list):
    genre_names_list = []
    for genre in genres_list:
        genre_names_list.append(genre['name'])
    return genre_names_list

games_df['genre'] = games_df['genre'].apply(lambda x:genre_names(x))

In [16]:
# define a function to get the ESRB rating names.
esrb = games_df['esrb_rating']

def esrb_label(esrb_dict):
    if esrb_dict != None:
        return esrb_dict['name']
    else:
        return np.NaN


games_df['esrb_rating'] = games_df['esrb_rating'].apply(lambda x: esrb_label(x))
    

In [105]:
games_df.head()

,id,title,release_date,tba,rating/5,top_rating,metacritic_score,average_length,platforms,genre,esrb_rating,genre_ids
0,3498,Grand Theft Auto V,2013-09-17,False,4.48,5,97.0,71,"[PlayStation 5, PlayStation 4, PlayStation 3, ...","[Action, Adventure]",Mature,"[4, 3]"
1,3328,The Witcher 3: Wild Hunt,2015-05-18,False,4.67,5,92.0,49,"[Nintendo Switch, Xbox One, PC, PlayStation 4]","[Action, Adventure, RPG]",Mature,"[4, 3, 5]"
2,4200,Portal 2,2011-04-18,False,4.62,5,95.0,11,"[Xbox 360, macOS, Linux, PC, PlayStation 3, Xb...","[Shooter, Puzzle]",Everyone 10+,"[2, 7]"
3,5286,Tomb Raider (2013),2013-03-05,False,4.06,4,86.0,11,"[macOS, Xbox One, PlayStation 4, PC, Xbox 360,...","[Action, Adventure]",Mature,"[4, 3]"
4,5679,The Elder Scrolls V: Skyrim,2011-11-11,False,4.43,5,94.0,45,"[PC, Nintendo Switch, Xbox 360, PlayStation 3]","[Action, RPG]",Mature,"[4, 5]"


In [18]:
games_df.sort_values(by='metacritic_score', ascending= False, inplace = True)
games_df.head(50)

,id,title,release_date,tba,rating/5,top_rating,metacritic_score,average_length,platforms,genre,esrb_rating,genre_ids
1473,25097,The Legend of Zelda: Ocarina of Time,1998-11-21,False,4.42,5,99.0,7,"[Nintendo 64, GameCube, Wii U, Wii]","[Action, Adventure, RPG]",Everyone,"[4, 3, 5]"
61829,407559,Soulcalibur (1998),1998-07-30,False,0.00,0,98.0,0,"[Dreamcast, Xbox 360]",[Fighting],Teen,[6]
6545,54751,Soulcalibur,1998-07-30,False,4.43,5,98.0,6,"[Xbox One, iOS, Xbox 360, Android, Dreamcast]","[Action, Fighting]",NaN,"[4, 6]"
0,3498,Grand Theft Auto V,2013-09-17,False,4.48,5,97.0,71,"[PlayStation 5, PlayStation 4, PlayStation 3, ...","[Action, Adventure]",Mature,"[4, 3]"
3958,56123,Metroid Prime,2002-11-17,False,4.38,5,97.0,2,"[GameCube, Wii]","[Action, Shooter, Adventure]",NaN,"[4, 2, 3]"
531,28026,Super Mario Odyssey,2017-10-26,False,4.44,5,97.0,0,[Nintendo Switch],"[Arcade, Platformer]",Everyone 10+,"[11, 83]"
265,22511,The Legend of Zelda: Breath of the Wild,2017-03-02,False,4.56,5,97.0,121,"[Wii U, Nintendo Switch]","[Action, Adventure, RPG]",Everyone 10+,"[4, 3, 5]"
1741,27024,Super Mario Galaxy,2007-11-12,False,4.40,5,97.0,28,"[Wii, Nintendo Switch, Wii U]",[Platformer],Everyone,[83]
2866,27036,Super Mario Galaxy 2,2010-05-23,False,4.37,5,97.0,18,"[Wii U, Wii]",[Platformer],NaN,[83]
2347,28495,Perfect Dark,2000-05-22,False,3.99,5,97.0,10,"[Xbox 360, Xbox One, Nintendo 64]",[Shooter],Mature,[2]


In [19]:
games_df.to_csv('games_data.csv')

In [20]:
games_df.isna().sum()

id                       0
title                    0
release_date          7725
tba                      0
rating/5                 0
top_rating               0
metacritic_score    194359
average_length           0
platforms                0
genre                    0
esrb_rating         180004
genre_ids                0
dtype: int64

In [22]:
cleaned_game_data = games_df.dropna()
cleaned_game_data.shape

(3099, 12)

In [26]:
cleaned_game_data = cleaned_game_data.sort_values(by='metacritic_score')
cleaned_game_data.to_csv('reduced_games_data.csv')